In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math

def no_pages(job):
    url = 'https://wuzzuf.net/search/jobs/?a=hpb&q=' + job.replace(' ', '%20') # means replace distance of job name by 20% to work dynamically
    response = requests.get(url)
    soup = BeautifulSoup(response.content,'lxml')  #the original page of a specific job
    
    max_num_jobs = int(soup.find('li', {'class': 'css-18k4nsw'}).text.split('of ')[1])
    pages = math.ceil(max_num_jobs/15)
    return pages, max_num_jobs

# to link between previous function and the following scrap, u must let the return values = previous function   
def scrap(job):
    num_pages, num_jobs = no_pages(job)  #new assignment names
    titles_list, links_list, occupations_list, companies_list, specs_list, locations_list = [], [], [], [], [], []
    for i in range(num_pages): # it is a loop to scrap all the pages of search results (not the first page only)
        job_to_link = 'https://wuzzuf.net/search/jobs/?a=hpb&q=' + job.replace(' ', '%20') # re-define the link in new function
        url = job_to_link + '&start=' + str(i) # the way of how link ia writien for each page
        page_link = requests.get(url)
        soup = BeautifulSoup(page_link.content, 'lxml') #the content of each page of the specific job (re-define)

        titles = soup.find_all('h2',{'class':'css-193uk2c'})
        titles_list += [title.a.text for title in titles]

        links_list += [title.a['href'] for title in titles]

        occupations = soup.find_all('div', {'class': 'css-5jhz9n'})
        occupations_list += [occupation.text for occupation in occupations]

        companies = soup.find_all("a",{'class': 'css-ipsyv7'})
        companies_list += [company.text for company in companies]

        specs = soup.find_all('div', {'class':'css-1rhj4yg'})
        specs_list += [spec.text for spec in specs]

        locations = soup.find_all('span',{'class': 'css-16x61xq'})
        locations_list += [location.text for location in locations]

    
    scraped_data = {
        'title': titles_list,
        'link' : links_list,
        'occupation' : occupations_list,
        'companie' : companies_list,
        'spec' : specs_list,
        'location': locations_list
    }
    table = pd.DataFrame(scraped_data)
    #table.to_csv('Sara_project3.csv', index = False)
    print('Jobs scraped sucessfully')
    #print(table.duplicated().sum()) #check if there is a repeat in rows for one job = 0
    return table

# if more than job as in Q: Repeat for : "data analysis" , "data science" , "business intelligence" then, Collect all in ( one CSV File )
# so 1- you must combine the 3 tables of these jobs in one table
def combine_dfs(tables):
    table = pd.concat(tables)
    table = table.drop_duplicates()
    return table

# then 2- you must combine eack key name of each list of the 3 dictionaries in one key at new combined dict
def combine_dicts(dicts):
    combined_dict = {} # empty final dict
    for key in dicts[0].keys():  # for loop the same key in each dic
        combined_dict[key] = [] # put this key in the empty key of the combined final
        for dict in dicts: # then for each dict of dicts
            combined_dict[key] += dict[key] # for key of final dict assign new addition for each elemnt dict at index key 
    return combined_dict   # exapmle final: { 'title': ['Job A', 'Job B', 'Job C'], 'location': ['Cairo', 'Giza', 'Alexandria']}

if __name__ == '__main__':
    jobs = ['data analysis', 'data science', 'business intelligence']
    tables = [scrap(job) for job in jobs]
    final_table = combine_dfs(tables)
    final_table.to_csv('all_jobs.csv', index=False)
    print('Jobs saved successfully to all_jobs.csv')
    print(final_table)

Jobs scraped sucessfully
Jobs scraped sucessfully
Jobs scraped sucessfully
Jobs saved successfully to all_jobs.csv
                                           title  \
0          Data analysis instructor / Consultant   
1    Data Analysis Instructor (Excel - Power BI)   
2                              Business Analysis   
3                          Data Analyst Engineer   
4                                   Data Analyst   
..                                           ...   
171            Full-Stack Engineer - Asia Region   
172                        Lead AI Engineer, UAE   
177         Software Engineer (iOS Core Product)   
178             Senior/Staff Engineer (Frontend)   
179                              Project Manager   

                                                  link  \
0    https://wuzzuf.net/jobs/p/wwl3ctsttqyz-data-an...   
1    https://wuzzuf.net/jobs/p/yrl7xpryqnne-data-an...   
2    https://wuzzuf.net/jobs/p/uuxdh2czgss6-busines...   
3    https://wuzzuf.net/jobs

In [19]:
scrap('Data Analyst')

Jobs scraped sucessfully


,title,link,occupation,companie,spec,location
0,Data Analyst,https://wuzzuf.net/jobs/p/yf6asb5mkpeg-data-an...,Full TimePart TimeOn-site,Confidential -,Full TimePart TimeOn-siteExperienced · 3 - 6 Y...,"Tora, Cairo, Egypt"
1,Data Analyst,https://wuzzuf.net/jobs/p/xyrvzfhuqygq-data-an...,Full TimeOn-site,Matana Group -,Full TimeOn-siteExperienced · 3 - 5 Yrs of Exp...,"Cairo, Egypt"
2,Data Analyst Engineer,https://wuzzuf.net/jobs/p/5lmb5l5ahjna-data-an...,Full TimeOn-site,Basco Engineering &Trading -,Full TimeOn-siteExperienced · 2 - 6 Yrs of Exp...,"Heliopolis, Cairo, Egypt"
3,Senior Data Analyst,https://wuzzuf.net/jobs/p/uwwoalbmat66-senior-...,Full TimeOn-site,Cairo Three A Poultry -,Full TimeOn-siteExperienced · 2 - 4 Yrs of Exp...,"New Cairo, Cairo, Egypt"
4,Data Analyst,https://wuzzuf.net/jobs/p/ymfo0bf34y0r-data-an...,Full TimeOn-site,Wills business -,Full TimeOn-siteExperienced · 2 - 6 Yrs of Exp...,"Nasr City, Cairo, Egypt"
...,...,...,...,...,...,...
220,Research Assistant,https://wuzzuf.net/jobs/p/g/sll7p1bk2m8d-resea...,Full Time,"United Arab Emirates University, Department of...",Full TimeEntry Level · 0 - 2 Yrs of Exp · Admi...,"Dubai, United Arab Emirates"
221,Head of Security Operations,https://wuzzuf.net/jobs/p/g/6g3pwtqvocmv-head-...,Full Time,AFNOYS -,Full TimeManager · 5 - 10 Yrs of Exp · IT/Soft...,"Dubai, United Arab Emirates"
222,Treasury & Bank Relations,https://wuzzuf.net/jobs/p/g/eppw3itr5pby-treas...,Full Time,Burjline Builders -,Full TimeExperienced · 5 - 10 Yrs of Exp · Acc...,"Riyadh, Saudi Arabia"
223,Senior Product Manager - Compliance,https://wuzzuf.net/jobs/p/g/hx1hypiza9jc-senio...,Full Time,Rapyd -,Full TimeExperienced · 5 - 10 Yrs of Exp · IT/...,"Dubai, United Arab Emirates"


In [20]:
scrap('data science')

Jobs scraped sucessfully


,title,link,occupation,companie,spec,location
0,Data Science Instructor,https://wuzzuf.net/jobs/p/f2mpxj1rzhp1-data-sc...,Full TimePart TimeFreelance / ProjectHybrid,EpsilonAI -,Full TimePart TimeFreelance / ProjectHybridEnt...,"Nasr City, Cairo, Egypt"
1,Senior Data Analyst,https://wuzzuf.net/jobs/p/8jyfd8nmzpfp-senior-...,Full TimeOn-site,Allianz -,Full TimeOn-siteExperienced · 2 - 3 Yrs of Exp...,"New Cairo, Cairo, Egypt"
2,Big Data Engineer,https://wuzzuf.net/jobs/p/izbdsxoaskyq-big-dat...,Full TimeHybrid,Efada Technology -,Full TimeHybridExperienced · 3 - 5 Yrs of Exp ...,"6th of October, Giza, Egypt"
3,Data Analysis Instructor (Excel - Power BI),https://wuzzuf.net/jobs/p/yrl7xpryqnne-data-an...,Full TimePart TimeFreelance / ProjectHybrid,EpsilonAI -,Full TimePart TimeFreelance / ProjectHybridEnt...,"Nasr City, Cairo, Egypt"
4,Data Analyst Engineer,https://wuzzuf.net/jobs/p/5lmb5l5ahjna-data-an...,Full TimeOn-site,Basco Engineering &Trading -,Full TimeOn-siteExperienced · 2 - 6 Yrs of Exp...,"Heliopolis, Cairo, Egypt"
...,...,...,...,...,...,...
530,Specialist - Quality Control,https://wuzzuf.net/jobs/p/g/sqic95rkrh2f-speci...,Full Time,Julphar -,Full TimeExperienced · 5 - 10 Yrs of Exp · Mar...,"Dubai, United Arab Emirates"
531,Enterprise Account Executive,https://wuzzuf.net/jobs/p/g/szesd1iglsyn-enter...,Full Time,Canonical -,Full TimeEntry Level · 5 - 10 Yrs of Exp · Cus...,"Riyadh, Saudi Arabia"
532,Senior Art Director,https://wuzzuf.net/jobs/p/g/ywcpilg9booc-senio...,Full Time,Monks -,Full TimeExperienced · 5 - 10 Yrs of Exp · Cre...,"Riyadh, Saudi Arabia"
533,Specialist - Project Controls,https://wuzzuf.net/jobs/p/g/z9lsewqlkeht-speci...,Full Time,AECOM -,Full TimeExperienced · 2 - 5 Yrs of Exp · Mark...,"Dubai, United Arab Emirates"


In [21]:
scrap('business intelligence')

Jobs scraped sucessfully


,title,link,occupation,companie,spec,location
0,Data Analyst,https://wuzzuf.net/jobs/p/xyrvzfhuqygq-data-an...,Full TimeOn-site,Matana Group -,Full TimeOn-siteExperienced · 3 - 5 Yrs of Exp...,"Cairo, Egypt"
1,Enterprise Data Warehouse and Business intelli...,https://wuzzuf.net/jobs/p/jchrh3unqyz6-enterpr...,Full TimeOn-site,"The Micro, Small & Medium Enterprise Developme...",Full TimeOn-siteEntry Level · 1 - 5 Yrs of Exp...,"Dokki, Giza, Egypt"
2,MIS Senior Specialist..,https://wuzzuf.net/jobs/p/x0zlfwdkeyqc-mis-sen...,Full TimeOn-site,Etisalat Egypt -,Full TimeOn-siteEntry Level · 1 - 3 Yrs of Exp...,"Maadi, Cairo, Egypt"
3,Senior Data Analyst,https://wuzzuf.net/jobs/p/uwwoalbmat66-senior-...,Full TimeOn-site,Cairo Three A Poultry -,Full TimeOn-siteExperienced · 2 - 4 Yrs of Exp...,"New Cairo, Cairo, Egypt"
4,Business Data Analyst,https://wuzzuf.net/jobs/p/tym1dyyj5cjs-busines...,Full TimeRemote,ENAYA CARE INTERNATIONAL COMPANY FOR BEAUTY & ...,Full TimeRemoteExperienced · 3 - 5 Yrs of Exp ...,"Cairo, Egypt"
...,...,...,...,...,...,...
175,"Specialist, Reservoir Engineering",https://wuzzuf.net/jobs/p/g/7va0klxz3v1m-speci...,Full Time,ADNOC Group -,Full TimeExperienced · 5 - 10 Yrs of Exp · Sal...,"Dubai, United Arab Emirates"
176,Professional Services Engineer AI ML,https://wuzzuf.net/jobs/p/g/naynoxio0zdu-profe...,Full Time,NVIDIA -,Full TimeExperienced · 2 - 5 Yrs of Exp · IT/S...,"Dubai, United Arab Emirates"
177,Software Engineer (iOS Core Product),https://wuzzuf.net/jobs/p/g/bo0twsm5vtfz-softw...,Full Time,Speechify -,Full TimeExperienced · 5 - 10 Yrs of Exp · IT/...,"Riyadh, Saudi Arabia"
178,Senior/Staff Engineer (Frontend),https://wuzzuf.net/jobs/p/g/wnt5s4ogbzx3-senio...,Full Time,AI71 -,Full TimeExperienced · 5 - 10 Yrs of Exp · IT/...,"Dubai, United Arab Emirates"


In [22]:
combine_dfs(tables)

,title,link,occupation,companie,spec,location
0,Data analysis instructor / Consultant,https://wuzzuf.net/jobs/p/wwl3ctsttqyz-data-an...,Full TimeOn-site,Yat Learning Centers -,Full TimeOn-siteExperienced · 3 - 16 Yrs of Ex...,"Abbassia, Cairo, Egypt"
1,Data Analysis Instructor (Excel - Power BI),https://wuzzuf.net/jobs/p/yrl7xpryqnne-data-an...,Full TimePart TimeFreelance / ProjectHybrid,EpsilonAI -,Full TimePart TimeFreelance / ProjectHybridEnt...,"Nasr City, Cairo, Egypt"
2,Business Analysis,https://wuzzuf.net/jobs/p/uuxdh2czgss6-busines...,Full TimeOn-site,KCG Textile Egypt -,Full TimeOn-siteExperienced · 2 - 3 Yrs of Exp...,"10th of Ramadan City, Sharqia, Egypt"
3,Data Analyst Engineer,https://wuzzuf.net/jobs/p/5lmb5l5ahjna-data-an...,Full TimeOn-site,Basco Engineering &Trading -,Full TimeOn-siteExperienced · 2 - 6 Yrs of Exp...,"Heliopolis, Cairo, Egypt"
4,Data Analyst,https://wuzzuf.net/jobs/p/yf6asb5mkpeg-data-an...,Full TimePart TimeOn-site,Confidential -,Full TimePart TimeOn-siteExperienced · 3 - 6 Y...,"Tora, Cairo, Egypt"
...,...,...,...,...,...,...
171,Full-Stack Engineer - Asia Region,https://wuzzuf.net/jobs/p/g/cyixnb7pqr0t-full-...,Full Time,The Flex -,Full TimeEntry Level · 2 - 5 Yrs of Exp · Crea...,"Riyadh, Saudi Arabia"
172,"Lead AI Engineer, UAE",https://wuzzuf.net/jobs/p/g/didv4jprfcva-lead-...,Full Time,ION -,Full TimeExperienced · 5 - 10 Yrs of Exp · IT/...,"Dubai, United Arab Emirates"
177,Software Engineer (iOS Core Product),https://wuzzuf.net/jobs/p/g/bo0twsm5vtfz-softw...,Full Time,Speechify -,Full TimeExperienced · 5 - 10 Yrs of Exp · IT/...,"Riyadh, Saudi Arabia"
178,Senior/Staff Engineer (Frontend),https://wuzzuf.net/jobs/p/g/wnt5s4ogbzx3-senio...,Full Time,AI71 -,Full TimeExperienced · 5 - 10 Yrs of Exp · IT/...,"Dubai, United Arab Emirates"
